In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pickle

FilePath = '/content/drive/MyDrive/ML-Project/weekly_final.csv'
# use dataset maker with numdays = 7
# reading csv file
DF = pd.read_csv(FilePath,delimiter=',')
# columns are features
columns = list(DF.columns)
# pre processing
for i in range(len(columns)):
  if "1_" in columns[i]:columns[i] = columns[i][2:]

DF.columns = columns

DF.drop(columns=['Unnamed: 0'], inplace=True)


columns = DF.columns
DF.head()


,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Month,Day,2_Location,2_MinTemp,2_MaxTemp,2_Rainfall,2_Evaporation,2_Sunshine,2_WindGustDir,2_WindGustSpeed,2_WindDir9am,2_WindDir3pm,2_WindSpeed9am,2_WindSpeed3pm,2_Humidity9am,2_Humidity3pm,2_Pressure9am,2_Pressure3pm,...,6_Pressure3pm,6_Cloud9am,6_Cloud3pm,6_Temp9am,6_Temp3pm,6_RainToday,6_Year,6_Month,6_Day,7_Location,7_MinTemp,7_MaxTemp,7_Rainfall,7_Evaporation,7_Sunshine,7_WindGustDir,7_WindGustSpeed,7_WindDir9am,7_WindDir3pm,7_WindSpeed9am,7_WindSpeed3pm,7_Humidity9am,7_Humidity3pm,7_Pressure9am,7_Pressure3pm,7_Cloud9am,7_Cloud3pm,7_Temp9am,7_Temp3pm,7_RainToday,7_Year,7_Month,7_Day,rainafter1days,rainafter2days,rainafter3days,rainafter4days,rainafter5days,rainafter6days,rainafter7days
0,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,NW,6.0,20.0,68.0,29.0,1019.7,1015.0,7.0,7.0,14.4,23.6,No,2007,11,1,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17.0,80.0,36.0,1012.4,1008.4,...,1021.7,7.0,5.0,10.9,14.8,No,2007,11,6,Canberra,6.1,18.2,0.2,4.2,8.4,SE,43.0,SE,ESE,19.0,26.0,63.0,47.0,1024.6,1022.2,4.0,6.0,12.4,17.3,No,2007,11,7,No,No,Yes,No,No,No,No
1,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17.0,80.0,36.0,1012.4,1008.4,5.0,3.0,17.5,25.7,Yes,2007,11,2,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6.0,82.0,69.0,1009.5,1007.2,...,1022.2,4.0,6.0,12.4,17.3,No,2007,11,7,Canberra,8.3,17.0,0.0,5.6,4.6,E,41.0,SE,E,11.0,24.0,65.0,57.0,1026.2,1024.2,6.0,7.0,12.1,15.5,No,2007,11,8,No,Yes,No,No,No,No,No
2,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6.0,82.0,69.0,1009.5,1007.2,8.0,7.0,15.4,20.2,Yes,2007,11,3,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24.0,62.0,56.0,1005.5,1007.0,...,1024.2,6.0,7.0,12.1,15.5,No,2007,11,8,Canberra,8.8,19.5,0.0,4.0,4.1,S,48.0,E,ENE,19.0,17.0,70.0,48.0,1026.1,1022.7,7.0,7.0,14.1,18.9,No,2007,11,9,Yes,No,No,No,No,No,No
3,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24.0,62.0,56.0,1005.5,1007.0,2.0,7.0,13.5,14.1,Yes,2007,11,4,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28.0,68.0,49.0,1018.3,1018.5,...,1022.7,7.0,7.0,14.1,18.9,No,2007,11,9,Canberra,8.4,22.8,16.2,5.4,7.7,E,31.0,S,ESE,7.0,6.0,82.0,32.0,1024.1,1020.7,7.0,1.0,13.3,21.7,Yes,2007,11,10,No,No,No,No,No,No,No
4,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28.0,68.0,49.0,1018.3,1018.5,7.0,7.0,11.1,15.4,Yes,2007,11,5,Canberra,6.2,16.9,0.0,5.8,8.2,SE,44.0,SE,E,20.0,24.0,70.0,57.0,1023.8,1021.7,...,1020.7,7.0,1.0,13.3,21.7,Yes,2007,11,10,Canberra,9.1,25.2,0.0,4.2,11.9,N,30.0,SE,NW,6.0,9.0,74.0,34.0,1024.4,1021.1,1.0,2.0,14.6,24.0,No,2007,11,11,No,No,No,No,No,No,Yes


In [ ]:
DF = DF.dropna(subset=['RainToday'])
c = 'RainToday'

# outputs
y1 = DF[['rainafter1days']]
y2 = DF[['rainafter2days']]
y3 = DF[['rainafter3days']]
y4 = DF[['rainafter4days']]
y5 = DF[['rainafter5days']]
y6 = DF[['rainafter6days']]
y7 = DF[['rainafter7days']]

#droping output columns from dataset
DF=DF.drop(columns = ['rainafter2days','rainafter2days','rainafter3days','rainafter4days','rainafter5days','rainafter6days','rainafter7days'])

# columns with non-object data type
numerical_col = [i for i in DF.columns if (DF[i].dtype=='float64' or DF[i].dtype=='int64')]
#columns with object data type
objects_col = [i for i in DF.columns if DF[i].dtype=='object']

# output vector
y=[y1,y2,y3,y4,y5,y6,y7]

# converting object type output column to int type.
for i in range(len(y)):
  y[i] = y[i].replace(to_replace={'Yes':1,'No':0})

# 6 models for 6 days
for i in range(6):

  # train test split
  X_train, X_test, y_train, y_test = train_test_split(DF, y[i], test_size=0.2, stratify=y[i], random_state=0)
  
  # PRE PROCESSING

  # taking median of non-object
  t = X_train.median()

  # filling nan values with median
  X_train = X_train.fillna(t)
  X_test = X_test.fillna(t)

  # filling object type nan values with mode
  for t_df in [X_train, X_test]:
    t_df['WindGustDir'].fillna(X_train['WindGustDir'].mode()[0], inplace=True)
    t_df['WindDir9am'].fillna(X_train['WindDir9am'].mode()[0], inplace=True)
    t_df['WindDir3pm'].fillna(X_train['WindDir3pm'].mode()[0], inplace=True)
    t_df['RainToday'].fillna(X_train['RainToday'].mode()[0], inplace=True)

  # one hot encoding for object type features
  X_train = pd.concat([X_train[numerical_col], pd.get_dummies(X_train.Location,prefix='Location'),
                      pd.get_dummies(X_train.WindGustDir,prefix='WindGustDir'),
                      pd.get_dummies(X_train.WindDir9am,prefix='WindDir9am'),
                      pd.get_dummies(X_train.WindDir3pm,prefix='WindDir3pm'),
                      pd.get_dummies(X_train.RainToday,prefix='RainToday')], axis=1)

  X_test = pd.concat([X_test[numerical_col], pd.get_dummies(X_test.Location,prefix='Location'),
                      pd.get_dummies(X_test.WindGustDir,prefix='WindGustDir'),
                      pd.get_dummies(X_test.WindDir9am,prefix='WindDir9am'),
                      pd.get_dummies(X_test.WindDir3pm,prefix='WindDir3pm'),
                      pd.get_dummies(X_test.RainToday,prefix='RainToday')], axis=1)
  
  #Standardizing data
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  accuracies = []
  
  # Random forest classifier
  clf = RandomForestClassifier(n_estimators=230, random_state=0,max_depth=22)

  clf.fit(X_train, y_train)
  pickle.dump(clf,open(f"/content/drive/MyDrive/ML-Project/1_day_{i+1}th_day.pickle","wb"))
  y_pred_test = clf.predict_proba(X_test)[:,1]
  
  # threshold value for classification
  thre = 0.5
  if i>0: thre = 0.38
  y_pred_test = np.where(y_pred_test>thre,1,0)

  # predicting 
  y_pred_train=clf.predict(X_train)
  test_accu = accuracy_score(y_test, y_pred_test)
  train_acc = accuracy_score(y_train,y_pred_train)
  accuracies.append([test_accu,train_acc])
  print('Day '+str(i+1),'test accuracy score : {0:0.4f}'. format(test_accu))
  print('Model train accuracy score : {0:0.4f}'. format(train_acc))

  #confussion matrix
  cm = confusion_matrix(y_test, y_pred_test)
  cm_matrix = pd.DataFrame(data=cm, columns=['Predict Negative:0', 'Predict Positive:1'], 
                                index=['Actual Negative:0', 'Actual positive:1'])
  display(cm_matrix)

v_c=y_test.value_counts()
v_c

null_accuracy = v_c[0]/sum(v_c)

print(null_accuracy)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 1 test accuracy score : 0.8572
Model train accuracy score : 0.9863


,Predict Negative:0,Predict Positive:1
Actual Negative:0,19130,622
Actual positive:1,2999,2613


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 2 test accuracy score : 0.8050
Model train accuracy score : 0.9777


...

,Predict Negative:0,Predict Positive:1
Actual Negative:0,19055,692
Actual positive:1,4469,1148


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 6 test accuracy score : 0.7953
Model train accuracy score : 0.9629


,Predict Negative:0,Predict Positive:1
Actual Negative:0,19019,724
Actual positive:1,4467,1154


rainafter6days
0                 0.778387
dtype: float64
